In [5]:
#Class 14 import thread
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.model_selection import cross_val_score, train_test_split, validation_curve
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor,export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [6]:
# Step 1 read and understand dataset
wine=pd.read_csv('../Data/winequality.csv')
wine.head(n=10)

# Clean data and add all column
### Data clean and pre-process
wine.dropna()
wine['type']=wine['type'].astype(str)
wine.rename(columns={'quality': 'quality_reg'}, inplace=True)
wine['quality_reg']=wine['quality_reg'].astype(int)
wine['quality_class'] = np.where(wine['quality_reg']>=6, 'good', 'bad')
wine.head(n=10)

#(Phase 1)REGRESSION run everything as a classification problem and then as classification of red and white
# (Phase 1.1)
## Create X, y sets, dummies and create test and train data
win, y_win= wine.iloc[:,:-2],wine['quality_reg']
win.head(n=10)
y_win.head(n=10)

# Run Regression to see if it works better
winc, y_winc= wine.iloc[:,:-2],wine['quality_class']
Xrg_win=pd.get_dummies(winc)
Xrg_wintr,Xrg_winte,yrg_wintr,yrg_winte=train_test_split(Xrg_win,y_win,random_state=0)

# (Phase 2)
wine.dropna()
subr=wine['type']=='red'
subw=wine['type']=='white'
winer=wine[subr]
winew=wine[subw]

# Red = Classification
# White = Classification
X_winr, y_winr= winer.iloc[:,1:-2],winer['quality_class']
X_winw, y_winw= winew.iloc[:,1:-2],winew['quality_class']

Xr_wintr,Xr_winte,yr_wintr,yr_winte=train_test_split(X_winr,y_winr,random_state=0)
Xw_wintr,Xw_winte,yw_wintr,yw_winte=train_test_split(X_winw,y_winw,random_state=0)

# Red2 = Regression (Incorrect)
# White2 = Classification
subr2=winer['quality_class']=='good'
winer2=winer[subr2]

subw2=winew['quality_class']=='good'
winew2=winew[subw2]
winew2['quality_class'] = np.where(winew2['quality_reg']>=7, 'good', 'bad')

X_winr2, y_winr2= winer2.iloc[:,1:-2],winer2['quality_reg']
X_winw2, y_winw2= winew2.iloc[:,1:-2],winew2['quality_class']

Xr_wintr2,Xr_winte2,yr_wintr2,yr_winte2=train_test_split(X_winr2,y_winr2,random_state=0)
Xw_wintr2,Xw_winte2,yw_wintr2,yw_winte2=train_test_split(X_winw2,y_winw2,random_state=0)

C:\Users\eraffo\Anaconda\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
# (Phase 1) Classification for good and bad
winc.head(n=10)
y_winc.head(n=10)

Xcl_win=pd.get_dummies(winc)
Xcl_win.head(n=10)

Xcl_wintr,Xcl_winte,ycl_wintr,ycl_winte=train_test_split(Xcl_win,y_winc,random_state=0)
Xcl_wintr.head(n=10)
ycl_wintr.head(n=10)
Xcl_winte.head(n=10)
ycl_winte.head(n=10)

5316    good
5210    good
3518     bad
1622    good
2443     bad
3521    good
5211     bad
4936     bad
1161    good
5993     bad
Name: quality_class, dtype: object

In [8]:
# understand shape of Model
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_reg
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525473,115.744728,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749313,56.521751,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [9]:
# Phase 1 on combined dataset
knn_lin1=KNeighborsClassifier(n_neighbors=1)
knn_lin1.fit(Xcl_wintr,ycl_wintr)
knn_lin2=KNeighborsClassifier(n_neighbors=3)
knn_lin2.fit(Xcl_wintr,ycl_wintr)
knn_lin3=KNeighborsClassifier(n_neighbors=4)
knn_lin3.fit(Xcl_wintr,ycl_wintr)
knn_lin4=KNeighborsClassifier(n_neighbors=5)
knn_lin4.fit(Xcl_wintr,ycl_wintr)
knn_lin5=KNeighborsClassifier(n_neighbors=6)
knn_lin5.fit(Xcl_wintr,ycl_wintr)
knn_lin6=KNeighborsClassifier(n_neighbors=10)
knn_lin6.fit(Xcl_wintr,ycl_wintr)

#Accuracy of Classification data
print('BP:  KNN1 acc on train: {:.3f}'.format(knn_lin1.score(Xcl_wintr,ycl_wintr)))
print('BP:  KNN1 acc on test: {:.3f}'.format(knn_lin1.score(Xcl_winte,ycl_winte)))
print('KNN2 acc on train: {:.3f}'.format(knn_lin2.score(Xcl_wintr,ycl_wintr)))
print('KNN2 acc on test: {:.3f}'.format(knn_lin2.score(Xcl_winte,ycl_winte)))
print('KNN3 acc on train: {:.3f}'.format(knn_lin3.score(Xcl_wintr,ycl_wintr)))
print('KNN3 acc on test: {:.3f}'.format(knn_lin3.score(Xcl_winte,ycl_winte)))
print('KNN4 acc on train: {:.3f}'.format(knn_lin4.score(Xcl_wintr,ycl_wintr)))
print('KNN4 acc on test: {:.3f}'.format(knn_lin4.score(Xcl_winte,ycl_winte)))
print('KNN5 acc on train: {:.3f}'.format(knn_lin5.score(Xcl_wintr,ycl_wintr)))
print('KNN5 acc on test: {:.3f}'.format(knn_lin5.score(Xcl_winte,ycl_winte)))
print('KNN6 acc on train: {:.3f}'.format(knn_lin6.score(Xcl_wintr,ycl_wintr)))
print('KNN6 acc on test: {:.3f}'.format(knn_lin6.score(Xcl_winte,ycl_winte)))

BP:  KNN1 acc on train: 1.000
BP:  KNN1 acc on test: 0.734
KNN2 acc on train: 0.854
KNN2 acc on test: 0.676
KNN3 acc on train: 0.810
KNN3 acc on test: 0.664
KNN4 acc on train: 0.796
KNN4 acc on test: 0.679
KNN5 acc on train: 0.776
KNN5 acc on test: 0.673
KNN6 acc on train: 0.741
KNN6 acc on test: 0.665


In [10]:
lor_lin=LogisticRegression()
lor_lin.fit(Xcl_wintr,ycl_wintr)

print('LOR acc on train: {:.3f}'.format(lor_lin.score(Xcl_wintr,ycl_wintr)))
print('LOR acc on test: {:.3f}'.format(lor_lin.score(Xcl_winte,ycl_winte)))

LOR acc on train: 0.742
LOR acc on test: 0.745


In [11]:
lsvm_lin1=LinearSVC(C=10,random_state=0)
lsvm_lin1.fit(Xcl_wintr,ycl_wintr)
lsvm_lin2=LinearSVC(C=1,random_state=0)
lsvm_lin2.fit(Xcl_wintr,ycl_wintr)
lsvm_lin3=LinearSVC(C=.1,random_state=0)
lsvm_lin3.fit(Xcl_wintr,ycl_wintr)
lsvm_lin4=LinearSVC(C=.01,random_state=0)
lsvm_lin4.fit(Xcl_wintr,ycl_wintr)
lsvm_lin5=LinearSVC(C=.001,random_state=0)
lsvm_lin5.fit(Xcl_wintr,ycl_wintr)

print('LSVM1 acc on train: {:.3f}'.format(lsvm_lin1.score(Xcl_wintr,ycl_wintr)))
print('LSVM1 acc on test: {:.3f}'.format(lsvm_lin1.score(Xcl_winte,ycl_winte)))
print('LSVM2 acc on train: {:.3f}'.format(lsvm_lin2.score(Xcl_wintr,ycl_wintr)))
print('LSVM2 acc on test: {:.3f}'.format(lsvm_lin2.score(Xcl_winte,ycl_winte)))
print('LSVM3 acc on train: {:.3f}'.format(lsvm_lin3.score(Xcl_wintr,ycl_wintr)))
print('LSVM3 acc on test: {:.3f}'.format(lsvm_lin3.score(Xcl_winte,ycl_winte)))
print('Best Predictor: LSVM4 acc on train: {:.3f}'.format(lsvm_lin4.score(Xcl_wintr,ycl_wintr)))
print('Best Predictor: LSVM4 acc on test: {:.3f}'.format(lsvm_lin4.score(Xcl_winte,ycl_winte)))
print('LSVM5 acc on train: {:.3f}'.format(lsvm_lin5.score(Xcl_wintr,ycl_wintr)))
print('LSVM5 acc on test: {:.3f}'.format(lsvm_lin5.score(Xcl_winte,ycl_winte)))

LSVM1 acc on train: 0.664
LSVM1 acc on test: 0.647
LSVM2 acc on train: 0.665
LSVM2 acc on test: 0.646
LSVM3 acc on train: 0.621
LSVM3 acc on test: 0.633
Best Predictor: LSVM4 acc on train: 0.727
Best Predictor: LSVM4 acc on test: 0.718
LSVM5 acc on train: 0.694
LSVM5 acc on test: 0.681


In [13]:
svm_lin1=SVC(gamma=10, random_state=0)
svm_lin1.fit(Xcl_wintr,ycl_wintr)
svm_lin2=SVC(gamma=1, random_state=0)
svm_lin2.fit(Xcl_wintr,ycl_wintr)
svm_lin3=SVC(gamma=.1, random_state=0)
svm_lin3.fit(Xcl_wintr,ycl_wintr)
svm_lin4=SVC(gamma=.01, random_state=0)
svm_lin4.fit(Xcl_wintr,ycl_wintr)
svm_lin5=SVC(gamma=.001, random_state=0)
svm_lin5.fit(Xcl_wintr,ycl_wintr)

print('SVM1 acc on train: {:.3f}'.format(svm_lin1.score(Xcl_wintr,ycl_wintr)))
print('SVM1 acc on test: {:.3f}'.format(svm_lin1.score(Xcl_winte,ycl_winte)))
print('SVM2 acc on train: {:.3f}'.format(svm_lin2.score(Xcl_wintr,ycl_wintr)))
print('SVM2 acc on test: {:.3f}'.format(svm_lin2.score(Xcl_winte,ycl_winte)))
print('Best Predictor: SVM3 acc on train: {:.3f}'.format(svm_lin3.score(Xcl_wintr,ycl_wintr)))
print('Best Predictor: SVM3 acc on test: {:.3f}'.format(svm_lin3.score(Xcl_winte,ycl_winte)))
print('SVM4 acc on train: {:.3f}'.format(svm_lin4.score(Xcl_wintr,ycl_wintr)))
print('SVM4 acc on test: {:.3f}'.format(svm_lin4.score(Xcl_winte,ycl_winte)))
print('SVM5 acc on train: {:.3f}'.format(svm_lin5.score(Xcl_wintr,ycl_wintr)))
print('SVM5 acc on test: {:.3f}'.format(svm_lin5.score(Xcl_winte,ycl_winte)))

SVM1 acc on train: 1.000
SVM1 acc on test: 0.715
SVM2 acc on train: 0.997
SVM2 acc on test: 0.714
Best Predictor: SVM3 acc on train: 0.912
Best Predictor: SVM3 acc on test: 0.725
SVM4 acc on train: 0.750
SVM4 acc on test: 0.698
SVM5 acc on train: 0.696
SVM5 acc on test: 0.655


In [14]:
dt_lin1=DecisionTreeClassifier(random_state=0)
dt_lin1.fit(Xcl_wintr,ycl_wintr)
dt_lin2=DecisionTreeClassifier(max_leaf_nodes=5,random_state=0)
dt_lin2.fit(Xcl_wintr,ycl_wintr)
dt_lin3=DecisionTreeClassifier(max_leaf_nodes=10,random_state=0)
dt_lin3.fit(Xcl_wintr,ycl_wintr)
dt_lin4=DecisionTreeClassifier(max_leaf_nodes=15,random_state=0)
dt_lin4.fit(Xcl_wintr,ycl_wintr)
dt_lin5=DecisionTreeClassifier(max_leaf_nodes=20,random_state=0)
dt_lin5.fit(Xcl_wintr,ycl_wintr)

print('DT1 acc on train: {:.3f}'.format(dt_lin1.score(Xcl_wintr,ycl_wintr)))
print('DT1 acc on test: {:.3f}'.format(dt_lin1.score(Xcl_winte,ycl_winte)))
print('DT2 acc on train: {:.3f}'.format(dt_lin2.score(Xcl_wintr,ycl_wintr)))
print('DT2 acc on test: {:.3f}'.format(dt_lin2.score(Xcl_winte,ycl_winte)))
print('DT3 acc on train: {:.3f}'.format(dt_lin3.score(Xcl_wintr,ycl_wintr)))
print('DT3 acc on test: {:.3f}'.format(dt_lin3.score(Xcl_winte,ycl_winte)))
print('Best Predictor: DT4 acc on train: {:.3f}'.format(dt_lin4.score(Xcl_wintr,ycl_wintr)))
print('Best Predictor: DT4 acc on test: {:.3f}'.format(dt_lin4.score(Xcl_winte,ycl_winte)))
print('DT5 acc on train: {:.3f}'.format(dt_lin5.score(Xcl_wintr,ycl_wintr)))
print('DT5 acc on test: {:.3f}'.format(dt_lin5.score(Xcl_winte,ycl_winte)))


DT1 acc on train: 1.000
DT1 acc on test: 0.783
DT2 acc on train: 0.733
DT2 acc on test: 0.742
DT3 acc on train: 0.747
DT3 acc on test: 0.716
Best Predictor: DT4 acc on train: 0.765
Best Predictor: DT4 acc on test: 0.738
DT5 acc on train: 0.769
DT5 acc on test: 0.735


In [15]:
rf_lin1=RandomForestClassifier(random_state=0,n_estimators=1)
rf_lin1.fit(Xcl_wintr,ycl_wintr)
rf_lin2=RandomForestClassifier(random_state=0,n_estimators=10)
rf_lin2.fit(Xcl_wintr,ycl_wintr)
rf_lin3=RandomForestClassifier(random_state=0,n_estimators=50)
rf_lin3.fit(Xcl_wintr,ycl_wintr)
rf_lin4=RandomForestClassifier(random_state=0,n_estimators=100)
rf_lin4.fit(Xcl_wintr,ycl_wintr)

print('RF1 acc on train: {:.3f}'.format(rf_lin1.score(Xcl_wintr,ycl_wintr)))
print('RF1 acc on test: {:.3f}'.format(rf_lin1.score(Xcl_winte,ycl_winte)))
print('RF2 acc on train: {:.3f}'.format(rf_lin2.score(Xcl_wintr,ycl_wintr)))
print('RF2 acc on test: {:.3f}'.format(rf_lin2.score(Xcl_winte,ycl_winte)))
print('Best Predictor: RF3 acc on train: {:.3f}'.format(rf_lin3.score(Xcl_wintr,ycl_wintr)))
print('Best Predictor: RF3 acc on test: {:.3f}'.format(rf_lin3.score(Xcl_winte,ycl_winte)))
print('RF4 acc on train: {:.3f}'.format(rf_lin4.score(Xcl_wintr,ycl_wintr)))
print('RF4 acc on test: {:.3f}'.format(rf_lin4.score(Xcl_winte,ycl_winte)))

RF1 acc on train: 0.897
RF1 acc on test: 0.742
RF2 acc on train: 0.991
RF2 acc on test: 0.812
Best Predictor: RF3 acc on train: 1.000
Best Predictor: RF3 acc on test: 0.825
RF4 acc on train: 1.000
RF4 acc on test: 0.823


In [16]:
class_importance=pd.DataFrame(data=rf_lin3.feature_importances_,
                                index=Xcl_win.columns, columns=['importance'])
print(class_importance.sort_values('importance',ascending=False))

                      importance
alcohol                 0.151985
volatile acidity        0.115596
density                 0.097970
chlorides               0.086533
free sulfur dioxide     0.085907
total sulfur dioxide    0.083196
sulphates               0.081278
residual sugar          0.079440
citric acid             0.075513
pH                      0.070515
fixed acidity           0.064437
type_white              0.003838
type_red                0.003790


In [17]:
print('KNN of 1 acc on train: {:.3f}'.format(knn_lin1.score(Xcl_wintr,ycl_wintr)))
print('KNN of 1 acc on test: {:.3f}'.format(knn_lin1.score(Xcl_winte,ycl_winte)))
print('LOR acc on train: {:.3f}'.format(lor_lin.score(Xcl_wintr,ycl_wintr)))
print('LOR acc on test: {:.3f}'.format(lor_lin.score(Xcl_winte,ycl_winte)))
print('LSVM C=.01 acc on train: {:.3f}'.format(lsvm_lin4.score(Xcl_wintr,ycl_wintr)))
print('LSVM C=.01 acc on test: {:.3f}'.format(lsvm_lin4.score(Xcl_winte,ycl_winte)))
print('SVM Gamma=.1 acc on train: {:.3f}'.format(svm_lin3.score(Xcl_wintr,ycl_wintr)))
print('SVM Gamma=.1acc on test: {:.3f}'.format(svm_lin3.score(Xcl_winte,ycl_winte)))
print('DT Leafs=15 acc on train: {:.3f}'.format(dt_lin4.score(Xcl_wintr,ycl_wintr)))
print('DT leafs=15 acc on test: {:.3f}'.format(dt_lin4.score(Xcl_winte,ycl_winte)))
print('RF of 50 acc on train: {:.3f}'.format(rf_lin3.score(Xcl_wintr,ycl_wintr)))
print('RF of 50 acc on test: {:.3f}'.format(rf_lin3.score(Xcl_winte,ycl_winte)))

KNN of 1 acc on train: 1.000
KNN of 1 acc on test: 0.734
LOR acc on train: 0.742
LOR acc on test: 0.745
LSVM C=.01 acc on train: 0.727
LSVM C=.01 acc on test: 0.718
SVM Gamma=.1 acc on train: 0.912
SVM Gamma=.1acc on test: 0.725
DT Leafs=15 acc on train: 0.765
DT leafs=15 acc on test: 0.738
RF of 50 acc on train: 1.000
RF of 50 acc on test: 0.825


In [18]:
winew['quality_reg'].describe() 

count    4898.000000
mean        5.877909
std         0.885639
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         9.000000
Name: quality_reg, dtype: float64

In [19]:
# RED = Classification
#Phase 2
knn_rlin1=KNeighborsClassifier(n_neighbors=15)
knn_rlin1.fit(Xr_wintr,yr_wintr)
knn_rlin2=KNeighborsClassifier(n_neighbors=20)
knn_rlin2.fit(Xr_wintr,yr_wintr)
knn_rlin3=KNeighborsClassifier(n_neighbors=25)
knn_rlin3.fit(Xr_wintr,yr_wintr)
knn_rlin4=KNeighborsClassifier(n_neighbors=30)
knn_rlin4.fit(Xr_wintr,yr_wintr)
knn_rlin5=KNeighborsClassifier(n_neighbors=35)
knn_rlin5.fit(Xr_wintr,yr_wintr)
knn_rlin6=KNeighborsClassifier(n_neighbors=40)
knn_rlin6.fit(Xr_wintr,yr_wintr)

#Accuracy of Classification data
print('KNN1 acc on train: {:.3f}'.format(knn_rlin1.score(Xr_wintr,yr_wintr)))
print('KNN1 acc on test: {:.3f}'.format(knn_rlin1.score(Xr_winte,yr_winte)))
print('KNN2 acc on train: {:.3f}'.format(knn_rlin2.score(Xr_wintr,yr_wintr)))
print('KNN2 acc on test: {:.3f}'.format(knn_rlin2.score(Xr_winte,yr_winte)))
print('KNN3 acc on train: {:.3f}'.format(knn_rlin3.score(Xr_wintr,yr_wintr)))
print('KNN3 acc on test: {:.3f}'.format(knn_rlin3.score(Xr_winte,yr_winte)))
print('Best Predictor: KNN of 30 acc on train: {:.3f}'.format(knn_rlin4.score(Xr_wintr,yr_wintr)))
print('Best Predictor: KNN of 30 acc on test: {:.3f}'.format(knn_rlin4.score(Xr_winte,yr_winte)))
print('KNN5 acc on train: {:.3f}'.format(knn_rlin5.score(Xr_wintr,yr_wintr)))
print('KNN5 acc on test: {:.3f}'.format(knn_rlin5.score(Xr_winte,yr_winte)))
print('KNN6 acc on train: {:.3f}'.format(knn_rlin6.score(Xr_wintr,yr_wintr)))
print('KNN6 acc on test: {:.3f}'.format(knn_rlin6.score(Xr_winte,yr_winte)))

KNN1 acc on train: 0.695
KNN1 acc on test: 0.677
KNN2 acc on train: 0.686
KNN2 acc on test: 0.682
KNN3 acc on train: 0.686
KNN3 acc on test: 0.682
Best Predictor: KNN of 30 acc on train: 0.681
Best Predictor: KNN of 30 acc on test: 0.688
KNN5 acc on train: 0.681
KNN5 acc on test: 0.672
KNN6 acc on train: 0.668
KNN6 acc on test: 0.647


In [20]:
lor_rlin=LogisticRegression()
lor_rlin.fit(Xr_wintr,yr_wintr)

print('LOR acc on train: {:.3f}'.format(lor_rlin.score(Xr_wintr,yr_wintr)))
print('LOR acc on test: {:.3f}'.format(lor_rlin.score(Xr_winte,yr_winte)))

LOR acc on train: 0.741
LOR acc on test: 0.757


In [21]:
lsvm_rlin1=LinearSVC(C=10,random_state=0)
lsvm_rlin1.fit(Xr_wintr,yr_wintr)
lsvm_rlin2=LinearSVC(C=1,random_state=0)
lsvm_rlin2.fit(Xr_wintr,yr_wintr)
lsvm_rlin3=LinearSVC(C=.1,random_state=0)
lsvm_rlin3.fit(Xr_wintr,yr_wintr)
lsvm_rlin4=LinearSVC(C=.01,random_state=0)
lsvm_rlin4.fit(Xr_wintr,yr_wintr)
lsvm_rlin5=LinearSVC(C=.001,random_state=0)
lsvm_rlin5.fit(Xr_wintr,yr_wintr)

print('LSVM1 acc on train: {:.3f}'.format(lsvm_rlin1.score(Xr_wintr,yr_wintr)))
print('LSVM1 acc on test: {:.3f}'.format(lsvm_rlin1.score(Xr_winte,yr_winte)))
print('LSVM2 acc on train: {:.3f}'.format(lsvm_rlin2.score(Xr_wintr,yr_wintr)))
print('LSVM2 acc on test: {:.3f}'.format(lsvm_rlin2.score(Xr_winte,yr_winte)))
print('Best Predictor: LSVM3 acc on train: {:.3f}'.format(lsvm_rlin3.score(Xr_wintr,yr_wintr)))
print('Best Predictor: LSVM3 acc on test: {:.3f}'.format(lsvm_rlin3.score(Xr_winte,yr_winte)))
print('LSVM4 acc on train: {:.3f}'.format(lsvm_rlin4.score(Xr_wintr,yr_wintr)))
print('LSVM4 acc on test: {:.3f}'.format(lsvm_rlin4.score(Xr_winte,yr_winte)))
print('LSVM5 acc on train: {:.3f}'.format(lsvm_rlin5.score(Xr_wintr,yr_wintr)))
print('LSVM5 acc on test: {:.3f}'.format(lsvm_rlin5.score(Xr_winte,yr_winte)))

LSVM1 acc on train: 0.546
LSVM1 acc on test: 0.552
LSVM2 acc on train: 0.569
LSVM2 acc on test: 0.580
Best Predictor: LSVM3 acc on train: 0.729
Best Predictor: LSVM3 acc on test: 0.745
LSVM4 acc on train: 0.726
LSVM4 acc on test: 0.740
LSVM5 acc on train: 0.660
LSVM5 acc on test: 0.665


In [22]:
svm_rlin1=SVC(gamma=10, random_state=0)
svm_rlin1.fit(Xr_wintr,yr_wintr)
svm_rlin2=SVC(gamma=1, random_state=0)
svm_rlin2.fit(Xr_wintr,yr_wintr)
svm_rlin3=SVC(gamma=.1, random_state=0)
svm_rlin3.fit(Xr_wintr,yr_wintr)
svm_rlin4=SVC(gamma=.01, random_state=0)
svm_rlin4.fit(Xr_wintr,yr_wintr)
svm_rlin5=SVC(gamma=.001, random_state=0)
svm_rlin5.fit(Xr_wintr,yr_wintr)

print('SVM1 acc on train: {:.3f}'.format(svm_rlin1.score(Xr_wintr,yr_wintr)))
print('SVM1 acc on test: {:.3f}'.format(svm_rlin1.score(Xr_winte,yr_winte)))
print('SVM2 acc on train: {:.3f}'.format(svm_rlin2.score(Xr_wintr,yr_wintr)))
print('SVM2 acc on test: {:.3f}'.format(svm_rlin2.score(Xr_winte,yr_winte)))
print('SVM3 acc on train: {:.3f}'.format(svm_rlin3.score(Xr_wintr,yr_wintr)))
print('SVM3 acc on test: {:.3f}'.format(svm_rlin3.score(Xr_winte,yr_winte)))
print('Best Prediction: SVM4 acc on train: {:.3f}'.format(svm_rlin4.score(Xr_wintr,yr_wintr)))
print('Best Prediction: SVM4 acc on test: {:.3f}'.format(svm_rlin4.score(Xr_winte,yr_winte)))
print('SVM5 acc on train: {:.3f}'.format(svm_rlin5.score(Xr_wintr,yr_wintr)))
print('SVM5 acc on test: {:.3f}'.format(svm_rlin5.score(Xr_winte,yr_winte)))

SVM1 acc on train: 1.000
SVM1 acc on test: 0.623
SVM2 acc on train: 0.996
SVM2 acc on test: 0.667
SVM3 acc on train: 0.872
SVM3 acc on test: 0.705
Best Prediction: SVM4 acc on train: 0.711
Best Prediction: SVM4 acc on test: 0.710
SVM5 acc on train: 0.646
SVM5 acc on test: 0.618


In [23]:
dt_rlin1=DecisionTreeClassifier(random_state=0)
dt_rlin1.fit(Xr_wintr,yr_wintr)
dt_rlin2=DecisionTreeClassifier(max_leaf_nodes=10,random_state=0)
dt_rlin2.fit(Xr_wintr,yr_wintr)
dt_rlin3=DecisionTreeClassifier(max_leaf_nodes=15,random_state=0)
dt_rlin3.fit(Xr_wintr,yr_wintr)
dt_rlin4=DecisionTreeClassifier(max_leaf_nodes=20,random_state=0)
dt_rlin4.fit(Xr_wintr,yr_wintr)
dt_rlin5=DecisionTreeClassifier(max_leaf_nodes=25,random_state=0)
dt_rlin5.fit(Xr_wintr,yr_wintr)

print('DT1 acc on train: {:.3f}'.format(dt_rlin1.score(Xr_wintr,yr_wintr)))
print('DT1 acc on test: {:.3f}'.format(dt_rlin1.score(Xr_winte,yr_winte)))
print('DT2 acc on train: {:.3f}'.format(dt_rlin2.score(Xr_wintr,yr_wintr)))
print('DT2 acc on test: {:.3f}'.format(dt_rlin2.score(Xr_winte,yr_winte)))
print('DT3 acc on train: {:.3f}'.format(dt_rlin3.score(Xr_wintr,yr_wintr)))
print('DT3 acc on test: {:.3f}'.format(dt_rlin3.score(Xr_winte,yr_winte)))
print('Best Predictor: DT4 acc on train: {:.3f}'.format(dt_rlin4.score(Xr_wintr,yr_wintr)))
print('Best Predictor: DT4 acc on test: {:.3f}'.format(dt_rlin4.score(Xr_winte,yr_winte)))
print('DT5 acc on train: {:.3f}'.format(dt_rlin5.score(Xr_wintr,yr_wintr)))
print('DT5 acc on test: {:.3f}'.format(dt_rlin5.score(Xr_winte,yr_winte)))


DT1 acc on train: 1.000
DT1 acc on test: 0.765
DT2 acc on train: 0.776
DT2 acc on test: 0.728
DT3 acc on train: 0.802
DT3 acc on test: 0.700
Best Predictor: DT4 acc on train: 0.820
Best Predictor: DT4 acc on test: 0.735
DT5 acc on train: 0.835
DT5 acc on test: 0.728


In [24]:
rf_rlin1=RandomForestClassifier(random_state=0,n_estimators=1)
rf_rlin1.fit(Xr_wintr,yr_wintr)
rf_rlin2=RandomForestClassifier(random_state=0,n_estimators=10)
rf_rlin2.fit(Xr_wintr,yr_wintr)
rf_rlin3=RandomForestClassifier(random_state=0,n_estimators=50)
rf_rlin3.fit(Xr_wintr,yr_wintr)
rf_rlin4=RandomForestClassifier(random_state=0,n_estimators=100)
rf_rlin4.fit(Xr_wintr,yr_wintr)

print('RF1 acc on train: {:.3f}'.format(rf_rlin1.score(Xr_wintr,yr_wintr)))
print('RF1 acc on test: {:.3f}'.format(rf_rlin1.score(Xr_winte,yr_winte)))
print('RF2 acc on train: {:.3f}'.format(rf_rlin2.score(Xr_wintr,yr_wintr)))
print('RF2 acc on test: {:.3f}'.format(rf_rlin2.score(Xr_winte,yr_winte)))
print('Best Predictor: RF3 acc on train: {:.3f}'.format(rf_rlin3.score(Xr_wintr,yr_wintr)))
print('Best Predictor: RF3 acc on test: {:.3f}'.format(rf_rlin3.score(Xr_winte,yr_winte)))
print('RF4 acc on train: {:.3f}'.format(rf_rlin4.score(Xr_wintr,yr_wintr)))
print('RF4 acc on test: {:.3f}'.format(rf_rlin4.score(Xr_winte,yr_winte)))

RF1 acc on train: 0.888
RF1 acc on test: 0.700
RF2 acc on train: 0.985
RF2 acc on test: 0.780
Best Predictor: RF3 acc on train: 1.000
Best Predictor: RF3 acc on test: 0.810
RF4 acc on train: 1.000
RF4 acc on test: 0.805


In [25]:
redclass_importance=pd.DataFrame(data=rf_rlin3.feature_importances_,
                                index=Xr_wintr.columns, columns=['importance'])
print(redclass_importance.sort_values('importance',ascending=False))

                      importance
alcohol                 0.183169
sulphates               0.137991
volatile acidity        0.108503
total sulfur dioxide    0.097881
density                 0.088560
pH                      0.068507
chlorides               0.068428
citric acid             0.066616
fixed acidity           0.066316
free sulfur dioxide     0.059956
residual sugar          0.054072


In [26]:
winew2.describe() 

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_reg
count,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000
mean,6.801059,0.262121,0.334131,6.057658,0.042921,35.292664,133.207489,0.993457,3.197231,0.494045,10.848880,6.382136
std,0.817327,0.090136,0.108314,4.929353,0.018262,15.138910,39.097152,0.003032,0.153517,0.120171,1.247434,0.594014
min,3.800000,0.080000,0.000000,0.700000,0.012000,3.000000,18.000000,0.987110,2.720000,0.220000,8.500000,6.000000
25%,6.300000,0.200000,0.270000,1.700000,0.034000,25.000000,105.000000,0.991182,3.090000,0.410000,9.800000,6.000000
50%,6.800000,0.250000,0.320000,4.750000,0.040000,34.000000,128.000000,0.992990,3.190000,0.480000,10.800000,6.000000
75%,7.300000,0.310000,0.380000,9.100000,0.048000,45.000000,158.000000,0.995420,3.300000,0.560000,11.900000,7.000000
max,14.200000,0.965000,1.660000,65.800000,0.255000,112.000000,294.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [27]:
# White = Classification
## Phase 2
knn_wlin1=KNeighborsClassifier(n_neighbors=1)
knn_wlin1.fit(Xw_wintr,yw_wintr)
knn_wlin2=KNeighborsClassifier(n_neighbors=5)
knn_wlin2.fit(Xw_wintr,yw_wintr)
knn_wlin3=KNeighborsClassifier(n_neighbors=10)
knn_wlin3.fit(Xw_wintr,yw_wintr)
knn_wlin4=KNeighborsClassifier(n_neighbors=14)
knn_wlin4.fit(Xw_wintr,yw_wintr)
knn_wlin5=KNeighborsClassifier(n_neighbors=16)
knn_wlin5.fit(Xw_wintr,yw_wintr)
knn_wlin6=KNeighborsClassifier(n_neighbors=20)
knn_wlin6.fit(Xw_wintr,yw_wintr)

#Accuracy of Classification data
print('KNN1 acc on train: {:.3f}'.format(knn_wlin1.score(Xw_wintr,yw_wintr)))
print('KNN1 acc on test: {:.3f}'.format(knn_wlin1.score(Xw_winte,yw_winte)))
print('KNN2 acc on train: {:.3f}'.format(knn_wlin2.score(Xw_wintr,yw_wintr)))
print('KNN2 acc on test: {:.3f}'.format(knn_wlin2.score(Xw_winte,yw_winte)))
print('Best Predictor: KNN3 acc on train: {:.3f}'.format(knn_wlin3.score(Xw_wintr,yw_wintr)))
print('Best Predictor: KNN3 acc on test: {:.3f}'.format(knn_wlin3.score(Xw_winte,yw_winte)))
print('KNN4 acc on train: {:.3f}'.format(knn_wlin4.score(Xw_wintr,yw_wintr)))
print('KNN4 acc on test: {:.3f}'.format(knn_wlin4.score(Xw_winte,yw_winte)))
print('KNN5 acc on train: {:.3f}'.format(knn_wlin5.score(Xw_wintr,yw_wintr)))
print('KNN5 acc on test: {:.3f}'.format(knn_wlin5.score(Xw_winte,yw_winte)))
print('KNN6 acc on train: {:.3f}'.format(knn_wlin6.score(Xw_wintr,yw_wintr)))
print('KNN6 acc on test: {:.3f}'.format(knn_wlin6.score(Xw_winte,yw_winte)))

KNN1 acc on train: 1.000
KNN1 acc on test: 0.724
KNN2 acc on train: 0.803
KNN2 acc on test: 0.677
Best Predictor: KNN3 acc on train: 0.760
Best Predictor: KNN3 acc on test: 0.682
KNN4 acc on train: 0.744
KNN4 acc on test: 0.664
KNN5 acc on train: 0.746
KNN5 acc on test: 0.660
KNN6 acc on train: 0.733
KNN6 acc on test: 0.642


In [51]:
lor_wlin=LogisticRegression()
lor_wlin.fit(Xw_wintr,yw_wintr)

print('LOR acc on train: {:.3f}'.format(lor_wlin.score(Xw_wintr,yw_wintr)))
print('LOR acc on test: {:.3f}'.format(lor_wlin.score(Xw_winte,yw_winte)))

LOR acc on train: 0.757
LOR acc on test: 0.732


In [29]:
lsvm_wlin1=LinearSVC(C=10,random_state=0)
lsvm_wlin1.fit(Xw_wintr,yw_wintr)
lsvm_wlin2=LinearSVC(C=1,random_state=0)
lsvm_wlin2.fit(Xw_wintr,yw_wintr)
lsvm_wlin3=LinearSVC(C=.1,random_state=0)
lsvm_wlin3.fit(Xw_wintr,yw_wintr)
lsvm_wlin4=LinearSVC(C=.01,random_state=0)
lsvm_wlin4.fit(Xw_wintr,yw_wintr)
lsvm_wlin5=LinearSVC(C=.001,random_state=0)
lsvm_wlin5.fit(Xw_wintr,yw_wintr)

print('LSVM1 acc on train: {:.3f}'.format(lsvm_wlin1.score(Xw_wintr,yw_wintr)))
print('LSVM1 acc on test: {:.3f}'.format(lsvm_wlin1.score(Xw_winte,yw_winte)))
print('LSVM2 acc on train: {:.3f}'.format(lsvm_wlin2.score(Xw_wintr,yw_wintr)))
print('LSVM2 acc on test: {:.3f}'.format(lsvm_wlin2.score(Xw_winte,yw_winte)))
print('LSVM3 acc on train: {:.3f}'.format(lsvm_wlin3.score(Xw_wintr,yw_wintr)))
print('LSVM3 acc on test: {:.3f}'.format(lsvm_wlin3.score(Xw_winte,yw_winte)))
print('Best Predictor: LSVM4 acc on train: {:.3f}'.format(lsvm_wlin4.score(Xw_wintr,yw_wintr)))
print('Best Predictor: LSVM4 acc on test: {:.3f}'.format(lsvm_wlin4.score(Xw_winte,yw_winte)))
print('LSVM5 acc on train: {:.3f}'.format(lsvm_wlin5.score(Xw_wintr,yw_wintr)))
print('LSVM5 acc on test: {:.3f}'.format(lsvm_wlin5.score(Xw_winte,yw_winte)))

LSVM1 acc on train: 0.562
LSVM1 acc on test: 0.595
LSVM2 acc on train: 0.714
LSVM2 acc on test: 0.682
LSVM3 acc on train: 0.727
LSVM3 acc on test: 0.690
Best Predictor: LSVM4 acc on train: 0.723
Best Predictor: LSVM4 acc on test: 0.705
LSVM5 acc on train: 0.689
LSVM5 acc on test: 0.652


In [30]:
svm_wlin1=SVC(gamma=10, random_state=0)
svm_wlin1.fit(Xw_wintr,yw_wintr)
svm_wlin2=SVC(gamma=1, random_state=0)
svm_wlin2.fit(Xw_wintr,yw_wintr)
svm_wlin3=SVC(gamma=.1, random_state=0)
svm_wlin3.fit(Xw_wintr,yw_wintr)
svm_wlin4=SVC(gamma=.01, random_state=0)
svm_wlin4.fit(Xw_wintr,yw_wintr)
svm_wlin5=SVC(gamma=.001, random_state=0)
svm_wlin5.fit(Xw_wintr,yw_wintr)

print('SVM1 acc on train: {:.3f}'.format(svm_wlin1.score(Xw_wintr,yw_wintr)))
print('SVM1 acc on test: {:.3f}'.format(svm_wlin1.score(Xw_winte,yw_winte)))
print('Best Predictor: SVM2 acc on train: {:.3f}'.format(svm_wlin2.score(Xw_wintr,yw_wintr)))
print('Best Predictor: SVM2 acc on test: {:.3f}'.format(svm_wlin2.score(Xw_winte,yw_winte)))
print('SVM3 acc on train: {:.3f}'.format(svm_wlin3.score(Xw_wintr,yw_wintr)))
print('SVM3 acc on test: {:.3f}'.format(svm_wlin3.score(Xw_winte,yw_winte)))
print('SVM4 acc on train: {:.3f}'.format(svm_wlin4.score(Xw_wintr,yw_wintr)))
print('SVM4 acc on test: {:.3f}'.format(svm_wlin4.score(Xw_winte,yw_winte)))
print('SVM5 acc on train: {:.3f}'.format(svm_wlin5.score(Xw_wintr,yw_wintr)))
print('SVM5 acc on test: {:.3f}'.format(svm_wlin5.score(Xw_winte,yw_winte)))

SVM1 acc on train: 1.000
SVM1 acc on test: 0.729
Best Predictor: SVM2 acc on train: 0.998
Best Predictor: SVM2 acc on test: 0.727
SVM3 acc on train: 0.927
SVM3 acc on test: 0.722
SVM4 acc on train: 0.776
SVM4 acc on test: 0.682
SVM5 acc on train: 0.718
SVM5 acc on test: 0.662


In [72]:
dt_wlin1=DecisionTreeClassifier(random_state=0)
dt_wlin1.fit(Xw_wintr,yw_wintr)
dt_wlin2=DecisionTreeClassifier(max_leaf_nodes=10,random_state=0)
dt_wlin2.fit(Xw_wintr,yw_wintr)
dt_wlin3=DecisionTreeClassifier(max_leaf_nodes=25,random_state=0)
dt_wlin3.fit(Xw_wintr,yw_wintr)
dt_wlin4=DecisionTreeClassifier(max_leaf_nodes=30,random_state=0)
dt_wlin4.fit(Xw_wintr,yw_wintr)
dt_wlin5=DecisionTreeClassifier(max_leaf_nodes=35,random_state=0)
dt_wlin5.fit(Xw_wintr,yw_wintr)

print('DT1 acc on train: {:.3f}'.format(dt_wlin1.score(Xw_wintr,yw_wintr)))
print('DT1 acc on test: {:.3f}'.format(dt_wlin1.score(Xw_winte,yw_winte)))
print('DT2 acc on train: {:.3f}'.format(dt_wlin2.score(Xw_wintr,yw_wintr)))
print('DT2 acc on test: {:.3f}'.format(dt_wlin2.score(Xw_winte,yw_winte)))
print('DT3 acc on train: {:.3f}'.format(dt_wlin3.score(Xw_wintr,yw_wintr)))
print('DT3 acc on test: {:.3f}'.format(dt_wlin3.score(Xw_winte,yw_winte)))
print('Best Predictor: DT4 acc on train: {:.3f}'.format(dt_wlin4.score(Xw_wintr,yw_wintr)))
print('Best Predictor: DT4 acc on test: {:.3f}'.format(dt_wlin4.score(Xw_winte,yw_winte)))
print('DT5 acc on train: {:.3f}'.format(dt_wlin5.score(Xw_wintr,yw_wintr)))
print('DT5 acc on test: {:.3f}'.format(dt_wlin5.score(Xw_winte,yw_winte)))

DT1 acc on train: 1.000
DT1 acc on test: 0.762
DT2 acc on train: 0.776
DT2 acc on test: 0.729
DT3 acc on train: 0.799
DT3 acc on test: 0.750
Best Predictor: DT4 acc on train: 0.804
Best Predictor: DT4 acc on test: 0.750
DT5 acc on train: 0.811
DT5 acc on test: 0.742


In [32]:
rf_wlin1=RandomForestClassifier(random_state=0,n_estimators=50)
rf_wlin1.fit(Xw_wintr,yw_wintr)
rf_wlin2=RandomForestClassifier(random_state=0,n_estimators=100)
rf_wlin2.fit(Xw_wintr,yw_wintr)
rf_wlin3=RandomForestClassifier(random_state=0,n_estimators=200)
rf_wlin3.fit(Xw_wintr,yw_wintr)
rf_wlin4=RandomForestClassifier(random_state=0,n_estimators=300)
rf_wlin4.fit(Xw_wintr,yw_wintr)

print('RF1 acc on train: {:.3f}'.format(rf_wlin1.score(Xw_wintr,yw_wintr)))
print('RF1 acc on test: {:.3f}'.format(rf_wlin1.score(Xw_winte,yw_winte)))
print('Best Predictor: RF2 acc on train: {:.3f}'.format(rf_wlin2.score(Xw_wintr,yw_wintr)))
print('Best Predictor: RF2 acc on test: {:.3f}'.format(rf_wlin2.score(Xw_winte,yw_winte)))
print('RF3 acc on train: {:.3f}'.format(rf_wlin3.score(Xw_wintr,yw_wintr)))
print('RF3 acc on test: {:.3f}'.format(rf_wlin3.score(Xw_winte,yw_winte)))
print('RF4 acc on train: {:.3f}'.format(rf_wlin4.score(Xw_wintr,yw_wintr)))
print('RF4 acc on test: {:.3f}'.format(rf_wlin4.score(Xw_winte,yw_winte)))

RF1 acc on train: 1.000
RF1 acc on test: 0.819
Best Predictor: RF2 acc on train: 1.000
Best Predictor: RF2 acc on test: 0.825
RF3 acc on train: 1.000
RF3 acc on test: 0.826
RF4 acc on train: 1.000
RF4 acc on test: 0.823


In [33]:
whiteclass_importance=pd.DataFrame(data=rf_wlin2.feature_importances_,
                                index=Xw_wintr.columns, columns=['importance'])
print(whiteclass_importance.sort_values('importance',ascending=False))

                      importance
alcohol                 0.143505
volatile acidity        0.115256
density                 0.107909
free sulfur dioxide     0.102896
total sulfur dioxide    0.086073
residual sugar          0.081336
citric acid             0.081118
chlorides               0.080260
pH                      0.071636
sulphates               0.065095
fixed acidity           0.064916


In [34]:
winer2.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_reg
count,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000,855.000000
mean,8.474035,0.474146,0.299883,2.535965,0.082661,15.273684,39.352047,0.996467,3.310643,0.692620,10.855029,6.274854
std,1.862795,0.161999,0.199889,1.424835,0.037258,10.039148,27.253280,0.002067,0.154554,0.155558,1.106109,0.491627
min,4.700000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.860000,0.390000,8.400000,6.000000
25%,7.100000,0.350000,0.115000,1.900000,0.067000,7.000000,20.000000,0.995185,3.210000,0.590000,10.000000,6.000000
50%,8.000000,0.460000,0.310000,2.200000,0.077000,13.000000,33.000000,0.996400,3.310000,0.660000,10.800000,6.000000
75%,9.650000,0.580000,0.460000,2.600000,0.087500,20.500000,50.000000,0.997685,3.400000,0.770000,11.700000,7.000000
max,15.600000,1.040000,0.780000,15.400000,0.415000,72.000000,289.000000,1.003690,4.010000,1.950000,14.000000,8.000000


In [35]:
# RED2 = Regression Models
# Incorrectly re-attempted a regression model
##NOT a regression problem
knn2_rreg=KNeighborsRegressor(n_neighbors=10)
knn2_rreg.fit(Xr_wintr2,yr_wintr2)

print('KNN4 acc on train: {:.3f}'.format(knn2_rreg.score(Xr_wintr2,yr_wintr2)))
print('KNN4 acc on test: {:.3f}'.format(knn2_rreg.score(Xr_winte2,yr_winte2)))

lin_rreg=LinearRegression()
lin_rreg.fit(Xr_wintr2,yr_wintr2)

print('LIN acc on train: {:.3f}'.format(lin_rreg.score(Xr_wintr2,yr_wintr2)))
print('LIN acc on test: {:.3f}'.format(lin_rreg.score(Xr_winte2,yr_winte2)))

log_rreg=LogisticRegression()
log_rreg.fit(Xr_wintr2,yr_wintr2)

print('LOG acc on train: {:.3f}'.format(log_rreg.score(Xr_wintr2,yr_wintr2)))
print('LOG acc on test: {:.3f}'.format(log_rreg.score(Xr_winte2,yr_winte2)))

las_rreg=Lasso(alpha=.001,random_state=0)
las_rreg.fit(Xr_wintr2,yr_wintr2)

print('Lasso acc on train: {:.3f}'.format(las_rreg.score(Xr_wintr2,yr_wintr2)))
print('Lasso acc on test: {:.3f}'.format(las_rreg.score(Xr_winte2,yr_winte2)))

rid_rreg=Ridge(alpha=.01, random_state=0)
rid_rreg.fit(Xr_wintr2,yr_wintr2)

print('Ridge acc on train: {:.3f}'.format(rid_rreg.score(Xr_wintr2,yr_wintr2)))
print('Ridge acc on test: {:.3f}'.format(rid_rreg.score(Xr_winte2,yr_winte2)))

dt_rreg=DecisionTreeRegressor(max_leaf_nodes=15,random_state=0)
dt_rreg.fit(Xr_wintr2,yr_wintr2)

print('DT acc on train: {:.3f}'.format(dt_rreg.score(Xr_wintr2,yr_wintr2)))
print('DT acc on test: {:.3f}'.format(dt_rreg.score(Xr_winte2,yr_winte2)))

rf_rreg=RandomForestRegressor(random_state=0,n_estimators=200)
rf_rreg.fit(Xr_wintr2,yr_wintr2)

print('RF acc on train: {:.3f}'.format(rf_rreg.score(Xr_wintr2,yr_wintr2)))
print('RF acc on test: {:.3f}'.format(rf_rreg.score(Xr_winte2,yr_winte2)))

KNN4 acc on train: 0.225
KNN4 acc on test: 0.038
LIN acc on train: 0.229
LIN acc on test: 0.157
LOG acc on train: 0.769
LOG acc on test: 0.771
Lasso acc on train: 0.215
Lasso acc on test: 0.155
Ridge acc on train: 0.222
Ridge acc on test: 0.156
DT acc on train: 0.406
DT acc on test: 0.080
RF acc on train: 0.906
RF acc on test: 0.330


In [38]:
winew2.describe() 

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_reg
count,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000,3258.000000
mean,6.801059,0.262121,0.334131,6.057658,0.042921,35.292664,133.207489,0.993457,3.197231,0.494045,10.848880,6.382136
std,0.817327,0.090136,0.108314,4.929353,0.018262,15.138910,39.097152,0.003032,0.153517,0.120171,1.247434,0.594014
min,3.800000,0.080000,0.000000,0.700000,0.012000,3.000000,18.000000,0.987110,2.720000,0.220000,8.500000,6.000000
25%,6.300000,0.200000,0.270000,1.700000,0.034000,25.000000,105.000000,0.991182,3.090000,0.410000,9.800000,6.000000
50%,6.800000,0.250000,0.320000,4.750000,0.040000,34.000000,128.000000,0.992990,3.190000,0.480000,10.800000,6.000000
75%,7.300000,0.310000,0.380000,9.100000,0.048000,45.000000,158.000000,0.995420,3.300000,0.560000,11.900000,7.000000
max,14.200000,0.965000,1.660000,65.800000,0.255000,112.000000,294.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [45]:
# White2 = Classification
# Phase 3 to see if we can make great wine
knn2_wlin1=KNeighborsClassifier(n_neighbors=1)
knn2_wlin1.fit(Xw_wintr2,yw_wintr2)
knn2_wlin2=KNeighborsClassifier(n_neighbors=5)
knn2_wlin2.fit(Xw_wintr2,yw_wintr2)
knn2_wlin3=KNeighborsClassifier(n_neighbors=10)
knn2_wlin3.fit(Xw_wintr2,yw_wintr2)
knn2_wlin4=KNeighborsClassifier(n_neighbors=14)
knn2_wlin4.fit(Xw_wintr2,yw_wintr2)
knn2_wlin5=KNeighborsClassifier(n_neighbors=16)
knn2_wlin5.fit(Xw_wintr2,yw_wintr2)
knn2_wlin6=KNeighborsClassifier(n_neighbors=20)
knn2_wlin6.fit(Xw_wintr2,yw_wintr2)

#Accuracy of Classification data
print('KNN1 acc on train: {:.3f}'.format(knn2_wlin1.score(Xw_wintr2,yw_wintr2)))
print('KNN1 acc on test: {:.3f}'.format(knn2_wlin1.score(Xw_winte2,yw_winte2)))
print('KNN2 acc on train: {:.3f}'.format(knn2_wlin2.score(Xw_wintr2,yw_wintr2)))
print('KNN2 acc on test: {:.3f}'.format(knn2_wlin2.score(Xw_winte2,yw_winte2)))
print('KNN3 acc on train: {:.3f}'.format(knn2_wlin3.score(Xw_wintr2,yw_wintr2)))
print('KNN3 acc on test: {:.3f}'.format(knn2_wlin3.score(Xw_winte2,yw_winte2)))
print('BP:  KNN4 acc on train: {:.3f}'.format(knn2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('BP:  KNN4 acc on test: {:.3f}'.format(knn2_wlin4.score(Xw_winte2,yw_winte2)))
print('KNN5 acc on train: {:.3f}'.format(knn2_wlin5.score(Xw_wintr2,yw_wintr2)))
print('KNN5 acc on test: {:.3f}'.format(knn2_wlin5.score(Xw_winte2,yw_winte2)))
print('KNN6 acc on train: {:.3f}'.format(knn2_wlin6.score(Xw_wintr2,yw_wintr2)))
print('KNN6 acc on test: {:.3f}'.format(knn2_wlin6.score(Xw_winte2,yw_winte2)))

KNN1 acc on train: 1.000
KNN1 acc on test: 0.702
KNN2 acc on train: 0.801
KNN2 acc on test: 0.683
KNN3 acc on train: 0.737
KNN3 acc on test: 0.686
BP:  KNN4 acc on train: 0.716
BP:  KNN4 acc on test: 0.693
KNN5 acc on train: 0.720
KNN5 acc on test: 0.682
KNN6 acc on train: 0.714
KNN6 acc on test: 0.685


In [52]:
lor2_wlin=LogisticRegression()
lor2_wlin.fit(Xw_wintr2,yw_wintr2)

print('LOR acc on train: {:.3f}'.format(lor2_wlin.score(Xw_wintr2,yw_wintr2)))
print('LOR acc on test: {:.3f}'.format(lor2_wlin.score(Xw_winte2,yw_winte2)))

LOR acc on train: 0.695
LOR acc on test: 0.719


In [56]:
lsvm2_wlin1=LinearSVC(C=10,random_state=0)
lsvm2_wlin1.fit(Xw_wintr2,yw_wintr2)
lsvm2_wlin2=LinearSVC(C=1,random_state=0)
lsvm2_wlin2.fit(Xw_wintr2,yw_wintr2)
lsvm2_wlin3=LinearSVC(C=.1,random_state=0)
lsvm2_wlin3.fit(Xw_wintr2,yw_wintr2)
lsvm2_wlin4=LinearSVC(C=.01,random_state=0)
lsvm2_wlin4.fit(Xw_wintr2,yw_wintr2)
lsvm2_wlin5=LinearSVC(C=.001,random_state=0)
lsvm2_wlin5.fit(Xw_wintr2,yw_wintr2)

print('LSVM1 acc on train: {:.3f}'.format(lsvm2_wlin1.score(Xw_wintr2,yw_wintr2)))
print('LSVM1 acc on test: {:.3f}'.format(lsvm2_wlin1.score(Xw_winte2,yw_winte2)))
print('LSVM2 acc on train: {:.3f}'.format(lsvm2_wlin2.score(Xw_wintr2,yw_wintr2)))
print('LSVM2 acc on test: {:.3f}'.format(lsvm2_wlin2.score(Xw_winte2,yw_winte2)))
print('LSVM3 acc on train: {:.3f}'.format(lsvm2_wlin3.score(Xw_wintr2,yw_wintr2)))
print('LSVM3 acc on test: {:.3f}'.format(lsvm2_wlin3.score(Xw_winte2,yw_winte2)))
print('Best Predictor: LSVM4 acc on train: {:.3f}'.format(lsvm2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('Best Predictor: LSVM4 acc on test: {:.3f}'.format(lsvm2_wlin4.score(Xw_winte2,yw_winte2)))
print('LSVM5 acc on train: {:.3f}'.format(lsvm2_wlin5.score(Xw_wintr2,yw_wintr2)))
print('LSVM5 acc on test: {:.3f}'.format(lsvm2_wlin5.score(Xw_winte2,yw_winte2)))

LSVM1 acc on train: 0.689
LSVM1 acc on test: 0.698
LSVM2 acc on train: 0.690
LSVM2 acc on test: 0.698
LSVM3 acc on train: 0.680
LSVM3 acc on test: 0.691
Best Predictor: LSVM4 acc on train: 0.693
Best Predictor: LSVM4 acc on test: 0.704
LSVM5 acc on train: 0.688
LSVM5 acc on test: 0.698


In [41]:
svm2_wlin1=SVC(gamma=10, random_state=0)
svm2_wlin1.fit(Xw_wintr2,yw_wintr2)
svm2_wlin2=SVC(gamma=1, random_state=0)
svm2_wlin2.fit(Xw_wintr2,yw_wintr2)
svm2_wlin3=SVC(gamma=.1, random_state=0)
svm2_wlin3.fit(Xw_wintr2,yw_wintr2)
svm2_wlin4=SVC(gamma=.01, random_state=0)
svm2_wlin4.fit(Xw_wintr2,yw_wintr2)
svm2_wlin5=SVC(gamma=.001, random_state=0)
svm2_wlin5.fit(Xw_wintr2,yw_wintr2)

print('SVM1 acc on train: {:.3f}'.format(svm2_wlin1.score(Xw_wintr2,yw_wintr2)))
print('SVM1 acc on test: {:.3f}'.format(svm2_wlin1.score(Xw_winte2,yw_winte2)))
print('BP: SVM2 acc on train: {:.3f}'.format(svm2_wlin2.score(Xw_wintr2,yw_wintr2)))
print('BP: SVM2 acc on test: {:.3f}'.format(svm2_wlin2.score(Xw_winte2,yw_winte2)))
print('SVM3 acc on train: {:.3f}'.format(svm2_wlin3.score(Xw_wintr2,yw_wintr2)))
print('SVM3 acc on test: {:.3f}'.format(svm2_wlin3.score(Xw_winte2,yw_winte2)))
print('SVM4 acc on train: {:.3f}'.format(svm2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('SVM4 acc on test: {:.3f}'.format(svm2_wlin4.score(Xw_winte2,yw_winte2)))
print('SVM5 acc on train: {:.3f}'.format(svm2_wlin5.score(Xw_wintr2,yw_wintr2)))
print('SVM5 acc on test: {:.3f}'.format(svm2_wlin5.score(Xw_winte2,yw_winte2)))

SVM1 acc on train: 1.000
SVM1 acc on test: 0.793
BP: SVM2 acc on train: 0.998
BP: SVM2 acc on test: 0.794
SVM3 acc on train: 0.913
SVM3 acc on test: 0.745
SVM4 acc on train: 0.742
SVM4 acc on test: 0.709
SVM5 acc on train: 0.672
SVM5 acc on test: 0.683


In [73]:
dt2_wlin1=DecisionTreeClassifier(random_state=0)
dt2_wlin1.fit(Xw_wintr2,yw_wintr2)
dt2_wlin2=DecisionTreeClassifier(max_leaf_nodes=10,random_state=0)
dt2_wlin2.fit(Xw_wintr2,yw_wintr2)
dt2_wlin3=DecisionTreeClassifier(max_leaf_nodes=25,random_state=0)
dt2_wlin3.fit(Xw_wintr2,yw_wintr2)
dt2_wlin4=DecisionTreeClassifier(max_leaf_nodes=30,random_state=0)
dt2_wlin4.fit(Xw_wintr2,yw_wintr2)
dt2_wlin5=DecisionTreeClassifier(max_leaf_nodes=35,random_state=0)
dt2_wlin5.fit(Xw_wintr2,yw_wintr2)

print('DT1 acc on train: {:.3f}'.format(dt2_wlin1.score(Xw_wintr2,yw_wintr2)))
print('DT1 acc on test: {:.3f}'.format(dt2_wlin1.score(Xw_winte2,yw_winte2)))
print('DT2 acc on train: {:.3f}'.format(dt2_wlin2.score(Xw_wintr2,yw_wintr2)))
print('DT2 acc on test: {:.3f}'.format(dt2_wlin2.score(Xw_winte2,yw_winte2)))
print('DT3 acc on train: {:.3f}'.format(dt2_wlin3.score(Xw_wintr2,yw_wintr2)))
print('DT3 acc on test: {:.3f}'.format(dt2_wlin3.score(Xw_winte2,yw_winte2)))
print('BP:  DT4 acc on train: {:.3f}'.format(dt2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('BP:  DT4 acc on test: {:.3f}'.format(dt2_wlin4.score(Xw_winte2,yw_winte2)))
print('DT5 acc on train: {:.3f}'.format(dt2_wlin5.score(Xw_wintr2,yw_wintr2)))
print('DT5 acc on test: {:.3f}'.format(dt2_wlin5.score(Xw_winte2,yw_winte2)))

DT1 acc on train: 1.000
DT1 acc on test: 0.775
DT2 acc on train: 0.740
DT2 acc on test: 0.740
DT3 acc on train: 0.761
DT3 acc on test: 0.729
BP:  DT4 acc on train: 0.770
BP:  DT4 acc on test: 0.739
DT5 acc on train: 0.782
DT5 acc on test: 0.737


In [64]:
rf2_wlin1=RandomForestClassifier(random_state=0,n_estimators=50)
rf2_wlin1.fit(Xw_wintr2,yw_wintr2)
rf2_wlin2=RandomForestClassifier(random_state=0,n_estimators=100)
rf2_wlin2.fit(Xw_wintr2,yw_wintr2)
rf2_wlin3=RandomForestClassifier(random_state=0,n_estimators=200)
rf2_wlin3.fit(Xw_wintr2,yw_wintr2)
rf2_wlin4=RandomForestClassifier(random_state=0,n_estimators=300)
rf2_wlin4.fit(Xw_wintr2,yw_wintr2)

print('RF1 acc on train: {:.3f}'.format(rf2_wlin1.score(Xw_wintr2,yw_wintr2)))
print('RF1 acc on test: {:.3f}'.format(rf2_wlin1.score(Xw_winte2,yw_winte2)))
print('RF2 acc on train: {:.3f}'.format(rf2_wlin2.score(Xw_wintr2,yw_wintr2)))
print('RF2 acc on test: {:.3f}'.format(rf2_wlin2.score(Xw_winte2,yw_winte2)))
print('RF3 acc on train: {:.3f}'.format(rf2_wlin3.score(Xw_wintr2,yw_wintr2)))
print('RF3 acc on test: {:.3f}'.format(rf2_wlin3.score(Xw_winte2,yw_winte2)))
print('BP:  RF4 acc on train: {:.3f}'.format(rf2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('BP:  RF4 acc on test: {:.3f}'.format(rf2_wlin4.score(Xw_winte2,yw_winte2)))

RF1 acc on train: 1.000
RF1 acc on test: 0.825
RF2 acc on train: 1.000
RF2 acc on test: 0.831
RF3 acc on train: 1.000
RF3 acc on test: 0.833
BP:  RF4 acc on train: 1.000
BP:  RF4 acc on test: 0.838


In [43]:
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_reg
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525473,115.744728,0.994697,3.218501,0.531268,10.491801,5.818378
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749313,56.521751,0.002999,0.160787,0.148806,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [46]:
#Whole Regression PHASE .01
## Decision = move to a classification model
knn_reg=KNeighborsRegressor(n_neighbors=15)
knn_reg.fit(Xrg_wintr,yrg_wintr)

print('KNN4 acc on train: {:.3f}'.format(knn_reg.score(Xrg_wintr,yrg_wintr)))
print('KNN4 acc on test: {:.3f}'.format(knn_reg.score(Xrg_winte,yrg_winte)))

lin=LinearRegression()
lin.fit(Xrg_wintr,yrg_wintr)

print('LIN acc on train: {:.3f}'.format(lin.score(Xrg_wintr,yrg_wintr)))
print('LIN acc on test: {:.3f}'.format(lin.score(Xrg_winte,yrg_winte)))

log=LogisticRegression()
log.fit(Xrg_wintr,yrg_wintr)

print('LOG acc on train: {:.3f}'.format(log.score(Xrg_wintr,yrg_wintr)))
print('LOG acc on test: {:.3f}'.format(log.score(Xrg_winte,yrg_winte)))

las=Lasso(alpha=.001,random_state=0)
las.fit(Xrg_wintr,yrg_wintr)

print('Lasso acc on train: {:.3f}'.format(las.score(Xrg_wintr,yrg_wintr)))
print('Lasso acc on test: {:.3f}'.format(las.score(Xrg_winte,yrg_winte)))

rid=Ridge(alpha=.001, random_state=0)
rid.fit(Xrg_wintr,yrg_wintr)

print('Ridge acc on train: {:.3f}'.format(rid.score(Xrg_wintr,yrg_wintr)))
print('Ridge acc on test: {:.3f}'.format(rid.score(Xrg_winte,yrg_winte)))

dt2=DecisionTreeRegressor(max_leaf_nodes=15,random_state=0)
dt2.fit(Xrg_wintr,yrg_wintr)

print('DT acc on train: {:.3f}'.format(dt2.score(Xrg_wintr,yrg_wintr)))
print('DT acc on test: {:.3f}'.format(dt2.score(Xrg_winte,yrg_winte)))

rf2=RandomForestRegressor(random_state=0,n_estimators=200)
rf2.fit(Xrg_wintr,yrg_wintr)

print('RF acc on train: {:.3f}'.format(rf2.score(Xrg_wintr,yrg_wintr)))
print('RF acc on test: {:.3f}'.format(rf2.score(Xrg_winte,yrg_winte)))

KNN4 acc on train: 0.269
KNN4 acc on test: 0.160
LIN acc on train: 0.297
LIN acc on test: 0.291
LOG acc on train: 0.541
LOG acc on test: 0.533
Lasso acc on train: 0.290
Lasso acc on test: 0.285
Ridge acc on train: 0.297
Ridge acc on test: 0.290
DT acc on train: 0.321
DT acc on test: 0.282
RF acc on train: 0.931
RF acc on test: 0.527


In [59]:
# For Paper
print('Whole: KNN of 1 acc on train: {:.3f}'.format(knn_lin1.score(Xcl_wintr,ycl_wintr)))
print('Whole: KNN of 1 acc on test: {:.3f}'.format(knn_lin1.score(Xcl_winte,ycl_winte)))
print('Red: KNN of 30 acc on train: {:.3f}'.format(knn_rlin4.score(Xr_wintr,yr_wintr)))
print('Red: KNN of 30 acc on test: {:.3f}'.format(knn_rlin4.score(Xr_winte,yr_winte)))
print('White: KNN of 10 acc on train: {:.3f}'.format(knn_wlin3.score(Xw_wintr,yw_wintr)))
print('White: KNN of 10 acc on test: {:.3f}'.format(knn_wlin3.score(Xw_winte,yw_winte)))
print('White 2: KNN of 14 on train: {:.3f}'.format(knn2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('White 2: KNN of 14 acc on test: {:.3f}'.format(knn2_wlin4.score(Xw_winte2,yw_winte2)))

Whole: KNN of 1 acc on train: 1.000
Whole: KNN of 1 acc on test: 0.734
Red: KNN of 30 acc on train: 0.681
Red: KNN of 30 acc on test: 0.688
White: KNN of 10 acc on train: 0.760
White: KNN of 10 acc on test: 0.682
White 2: KNN of 14 on train: 0.716
White 2: KNN of 14 acc on test: 0.693


In [54]:
print('Whole: LOR acc on train: {:.3f}'.format(lor_lin.score(Xcl_wintr,ycl_wintr)))
print('Whole: LOR acc on test: {:.3f}'.format(lor_lin.score(Xcl_winte,ycl_winte)))
print('Red: LOR acc on train: {:.3f}'.format(lor_rlin.score(Xr_wintr,yr_wintr)))
print('Red: LOR acc on test: {:.3f}'.format(lor_rlin.score(Xr_winte,yr_winte)))
print('White: LOR acc on train: {:.3f}'.format(lor_wlin.score(Xw_wintr,yw_wintr)))
print('White: LOR acc on test: {:.3f}'.format(lor_wlin.score(Xw_winte,yw_winte)))
print('White 2: LOR acc on train: {:.3f}'.format(lor2_wlin.score(Xw_wintr2,yw_wintr2)))
print('White 2: LOR acc on test: {:.3f}'.format(lor2_wlin.score(Xw_winte2,yw_winte2)))

Whole: LOR acc on train: 0.742
Whole: LOR acc on test: 0.745
Red: LOR acc on train: 0.741
Red: LOR acc on test: 0.757
White: LOR acc on train: 0.757
White: LOR acc on test: 0.732
White 2: LOR acc on train: 0.695
White 2: LOR acc on test: 0.719


In [58]:
print('Whole: LSVM of C=.01 acc on train: {:.3f}'.format(lsvm_lin4.score(Xcl_wintr,ycl_wintr)))
print('Whole: LSVM of C=.01 acc on test: {:.3f}'.format(lsvm_lin4.score(Xcl_winte,ycl_winte)))
print('Red: LSVM of C=.1 acc on train: {:.3f}'.format(lsvm_rlin3.score(Xr_wintr,yr_wintr)))
print('Red: LSVM of C=.1 acc on test: {:.3f}'.format(lsvm_rlin3.score(Xr_winte,yr_winte)))
print('White: LSVM of C=.01 acc on train: {:.3f}'.format(lsvm_wlin4.score(Xw_wintr,yw_wintr)))
print('White: LSVM of C=.01 acc on test: {:.3f}'.format(lsvm_wlin4.score(Xw_winte,yw_winte)))
print('White 2: LSVM of C=.01 acc on train: {:.3f}'.format(lsvm2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('White 2: LSVM of C=.01 acc on test: {:.3f}'.format(lsvm2_wlin4.score(Xw_winte2,yw_winte2)))

Whole: LSVM of C=.01 acc on train: 0.727
Whole: LSVM of C=.01 acc on test: 0.718
Red: LSVM of C=.1 acc on train: 0.729
Red: LSVM of C=.1 acc on test: 0.745
White: LSVM of C=.01 acc on train: 0.723
White: LSVM of C=.01 acc on test: 0.705
White 2: LSVM of C=.01 acc on train: 0.693
White 2: LSVM of C=.01 acc on test: 0.704


In [61]:
print('Whole: SVM g=.1 acc on train: {:.3f}'.format(svm_lin3.score(Xcl_wintr,ycl_wintr)))
print('Whole: SVM g=.1 acc on test: {:.3f}'.format(svm_lin3.score(Xcl_winte,ycl_winte)))
print('Red: SVM g=.01 acc on train: {:.3f}'.format(svm_rlin4.score(Xr_wintr,yr_wintr)))
print('Red: SVM g=.01 acc on test: {:.3f}'.format(svm_rlin4.score(Xr_winte,yr_winte)))
print('White: SVM g=1 acc on train: {:.3f}'.format(svm_wlin2.score(Xw_wintr,yw_wintr)))
print('White: SVM g=1 acc on test: {:.3f}'.format(svm_wlin2.score(Xw_winte,yw_winte)))
print('White 2: SVM g=1 acc on train: {:.3f}'.format(svm2_wlin2.score(Xw_wintr2,yw_wintr2)))
print('White 2: SVM g=1 acc on test: {:.3f}'.format(svm2_wlin2.score(Xw_winte2,yw_winte2)))

Whole: SVM g=.1 acc on train: 0.912
Whole: SVM g=.1 acc on test: 0.725
Red: SVM g=.01 acc on train: 0.711
Red: SVM g=.01 acc on test: 0.710
White: SVM g=1 acc on train: 0.998
White: SVM g=1 acc on test: 0.727
White 2: SVM g=1 acc on train: 0.998
White 2: SVM g=1 acc on test: 0.794


In [66]:
print('Whole: DT of 15 LN acc on train: {:.3f}'.format(dt_lin4.score(Xcl_wintr,ycl_wintr)))
print('Whole: DT of 15 LN acc on test: {:.3f}'.format(dt_lin4.score(Xcl_winte,ycl_winte)))
print('Red: DT of 20 LN acc on train: {:.3f}'.format(dt_rlin4.score(Xr_wintr,yr_wintr)))
print('Red: DT of 20 LN acc on test: {:.3f}'.format(dt_rlin4.score(Xr_winte,yr_winte)))
print('White: DT of 30 LN acc on train: {:.3f}'.format(dt_wlin4.score(Xw_wintr,yw_wintr)))
print('White: DT of 30 LN on test: {:.3f}'.format(dt_wlin4.score(Xw_winte,yw_winte)))
print('White 2: DT of 30 LN acc on train: {:.3f}'.format(dt2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('White 2: DT of 30 LN acc on test: {:.3f}'.format(dt2_wlin4.score(Xw_winte2,yw_winte2)))

Whole: DT of 15 LN acc on train: 0.765
Whole: DT of 15 LN acc on test: 0.738
Red: DT of 20 LN acc on train: 0.820
Red: DT of 20 LN acc on test: 0.735
White: DT of 30 LN acc on train: 0.804
White: DT of 30 LN on test: 0.750
White 2: DT of 30 LN acc on train: 0.770
White 2: DT of 30 LN acc on test: 0.739


In [71]:
print('Whole: RF of 50 acc on train: {:.3f}'.format(rf_lin3.score(Xcl_wintr,ycl_wintr)))
print('Whole: RF of 50 acc on test: {:.3f}'.format(rf_lin3.score(Xcl_winte,ycl_winte)))
print('Red: RF of 50 acc on train: {:.3f}'.format(rf_rlin3.score(Xr_wintr,yr_wintr)))
print('Red: RF of 50 acc on test: {:.3f}'.format(rf_rlin3.score(Xr_winte,yr_winte)))
print('White: RF of 100 acc on train: {:.3f}'.format(rf_wlin2.score(Xw_wintr,yw_wintr)))
print('White: RF of 100 acc on test: {:.3f}'.format(rf_wlin2.score(Xw_winte,yw_winte)))
print('White 2: RF of 300 acc on train: {:.3f}'.format(rf2_wlin4.score(Xw_wintr2,yw_wintr2)))
print('White 2: RF of 300 acc on test: {:.3f}'.format(rf2_wlin4.score(Xw_winte2,yw_winte2)))

Whole: RF of 50 acc on train: 1.000
Whole: RF of 50 acc on test: 0.825
Red: RF of 50 acc on train: 1.000
Red: RF of 50 acc on test: 0.810
White: RF of 100 acc on train: 1.000
White: RF of 100 acc on test: 0.825
White 2: RF of 300 acc on train: 1.000
White 2: RF of 300 acc on test: 0.838


In [74]:
# Decision Tree visualization 
# White
export_graphviz(dt_wlin2,out_file='dtwwine_EXvis.dot',class_names=dt_wlin2.classes_,
                feature_names=X_winw.columns,impurity=False,filled=True)
export_graphviz(dt_wlin4,out_file='dtwwine_BPvis.dot',class_names=dt_wlin4.classes_,
                feature_names=X_winw.columns,impurity=False,filled=True)
#Red
export_graphviz(dt_rlin2,out_file='dtrwine_EXvis.dot',class_names=dt_rlin2.classes_,
                feature_names=X_winr.columns,impurity=False,filled=True)
export_graphviz(dt_rlin4,out_file='dtrwine_BPvis.dot',class_names=dt_rlin4.classes_,
                feature_names=X_winr.columns,impurity=False,filled=True)
#Whole
export_graphviz(dt_lin3,out_file='dtwine_EXvis.dot',class_names=dt_lin3.classes_,
                feature_names=Xcl_win.columns,impurity=False,filled=True)
export_graphviz(dt_lin4,out_file='dtwine_BPvis.dot',class_names=dt_lin4.classes_,
                feature_names=Xcl_win.columns,impurity=False,filled=True)
#White 2
export_graphviz(dt2_wlin2,out_file='dtw2wine_EXvis.dot',class_names=dt2_wlin2.classes_,
                feature_names=X_winw.columns,impurity=False,filled=True)
export_graphviz(dt2_wlin4,out_file='dtw2wine_BPvis.dot',class_names=dt2_wlin4.classes_,
                feature_names=X_winw2.columns,impurity=False,filled=True)